# **8. 다중 입출력, 병렬형 아키텍처**

- Functional API 이용
- 연관성 있는 정보는 각각 모델을 돌리는 것보다 한번에 넣어 **결합분포의 효과**를 기대하는것이 좋음

### 1) 다중 입력 아키텍쳐

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, GlobalMaxPooling2D, Embedding, LSTM, Dense, concatenate
from tensorflow.keras import Input

## 이미지
image_input=Input(shape=(128,128,3))
image_layer=Conv2D(64, (3,3), activation='relu')(image_input)
image_pool=GlobalMaxPooling2D()(image_layer)  # 병목현상이 발생하는걸 막음 

## 텍스트 
maxlen=500  # 500개의 단어가 들어옴 (timestemp를 500개로 잡음)
text_input=Input(shape=(maxlen,))
text_layer=Embedding(10000,120)(text_input)
text_lstm=LSTM(32)(text_layer)

## 나이
age_input=Input(shape=(1,))
age_layer=Dense(5)(age_input)

concatenated=concatenate([image_pool, text_lstm, age_layer], axis=-1)   # 세가지 입력에서 온 레이어를 concat 가능
price=Dense(1)(concatenated)

model=Model([image_input, text_input, age_input], price)
model.summary()

model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
model.fit([image, text, age], y_train, epochs=20, batch_size=128)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 500)]        0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 126, 126, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
embedding (Embedding)           (None, 500, 120)     1200000     input_2[0][0]                    
______________________________________________________________________________________________

In [ ]:
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
model.fit([image, text, age], y_train, epochs=20, batch_size=128)

### 2) 다중 출력 아키텍처 

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, Embedding, Dense
from tensorflow.keras import Input


maxlen=500
text_input=Input(shape=(maxlen,))
embedding_text=Embedding(10000,128)(text_input)  # 임베딩은 MLP 모형임(10000개의 onehot을 dense representation)
x=Conv1D(256,7,activation='relu')(embedding_text)  # timestemp를 conv1d로 만들어냄
x=MaxPooling1D(5)(x)

x=Conv1D(512,5,activation='relu')(x)
x=GlobalMaxPooling1D()(x)

x=Dense(128,activation='relu')(x)

sex_out=Dense(1, activation='sigmoid', name='sex')(x)  # name을 넣으면 레이어가 잘 구분됨
age_out=Dense(1,name='age')(x)
job_out=Dense(10,activation='softmax', name='job')(x)

model=Model(text_input, [sex_out,age_out,job_out])   # 다중출력 
model.summary()

model.compile(optimizer='rmsprop', loss=['binary_crossentropy','mse','categorical_crossentropy'])  # loss가 세개
## loss의 합을 줄임(loss의 weight를 준다)